In [ ]:
# 데이터 Split
X_train, X_test, Y_train, Y_test = train_test_split(f_pcm_df[f_pcm_df.columns[19:]].values, f_pcm_df[['AVG']].values, test_size=0.1, random_state=321, shuffle=False)

# min max scailing
x_scaler = MinMaxScaler(feature_range=(1,100))
x_scaler.fit(X_train)
X_train = x_scaler.transform(X_train)
X_test = x_scaler.transform(X_test)

y_scaler = MinMaxScaler(feature_range=(1,100))
y_scaler.fit(Y_train)
Y_train = y_scaler.transform(Y_train)
Y_test = y_scaler.transform(Y_test)

# Feature Selection
model_LR = RandomForestRegressor(random_state=777)
select_RFE = RFE(model_LR, n_features_to_select=10)
X_train_RFE = select_RFE.fit(X_train,Y_train)
print(f_pcm_df.columns[19:][X_train_RFE.support_])

In [ ]:
# 모뎅 생성 및 평가
# 상기 내용과 거의 일치
def make_n_val_model(pcm_df, iqc_df, route_list, x_list, target):
    total_eq_df = pd.DataFrame(pcm_df['LOT_NO'], columns = ['LOT_NO'])
    for item1 in route_list:
        step = iqc_df[iqc_df['ROUTE']== item1]
        for item2 in sorted(set(step['PARA'])):
            if item2[:3] == 'MCD' or item2[:3] == 'DCD' or item2[:3] == 'RCD':
                continue
            step2 = step[step['PARA'] == item2]
            vals = [f'NO_VAL{i+1}'for i in range(int(step2['NO_VAL'].max()))]
            step2_pivot = pd.pivot_table(step2[['DTTM', 'LPT', 'LOT_NO','MAIN_EQ', 'UNIT']+vals],
                   index=['DTTM', 'LPT', 'LOT_NO', 'MAIN_EQ'],
                   columns='UNIT',
                   values=vals,
                   aggfunc='last'
                  )
            step2_pivot[f'{item1}_{item2}_AVG'] = step2_pivot.mean(axis=1)
            step2_pivot[f'{item1}_{item2}_MAX'] = step2_pivot.max(axis=1)
            step2_pivot[f'{item1}_{item2}_MIN'] = step2_pivot.min(axis=1)

            step2_pivot = step2_pivot.reset_index()[['DTTM', 'LOT_NO', f'{item1}_{item2}_AVG',f'{item1}_{item2}_MAX',f'{item1}_{item2}_MIN']]
            step2_pivot.columns = step2_pivot.columns.droplevel(1)

            step2_pivot.sort_values(by=['LOT_NO', 'DTTM'], ascending=False, inplace=True)
            step2_pivot['rn'] = step2_pivot.groupby(['LOT_NO']).cumcount() + 1

            step2_pivot = step2_pivot[step2_pivot['rn'] == 1]

            step2_pivot = step2_pivot.drop(['DTTM', 'rn'], axis=1)

            pcm_df = pd.merge(pcm_df, step2_pivot, how='left', left_on='LOT_NO', right_on='LOT_NO')

        if f'{item1}_CD15AS_AVG' in pcm_df.columns and f'{item1}_SCD15AS_AVG' in pcm_df.columns:
            pcm_df[f'{item1}_CD15AS_AVG'] = pcm_df[f'{item1}_CD15AS_AVG'].fillna(pcm_df[f'{item1}_SCD15AS_AVG'])
            pcm_df[f'{item1}_CD15AS_MAX'] = pcm_df[f'{item1}_CD15AS_MAX'].fillna(pcm_df[f'{item1}_SCD15AS_MAX'])
            pcm_df[f'{item1}_CD15AS_MIN'] = pcm_df[f'{item1}_CD15AS_MIN'].fillna(pcm_df[f'{item1}_SCD15AS_MIN'])
            pcm_df = pcm_df.drop([f'{item1}_SCD15AS_AVG', f'{item1}_SCD15AS_MAX', f'{item1}_SCD15AS_MIN'], axis=1)
        
        ohe = OneHotEncoder(sparse=False)
        ohe.fit(step[['MAIN_EQ']])
        eq_df = pd.DataFrame(ohe.transform(step[['MAIN_EQ']]), columns=list(step['MAIN_EQ'].unique()), index = step['LOT_NO']).reset_index().drop_duplicates()

    pcm_df = pd.merge(pcm_df, total_eq_df, how='left')
    pcm_df = pcm_df.fillna(method='ffill')
    pcm_df = pcm_df.fillna(method='bfill')
    pcm_df = pcm_df.dropna()
    print(len(pcm_df), len(pcm_df.columns))
    print('data prepare')
    
    total_eq_df.drop('LOT_NO',axis=1, inplace=True)
    
    X_train, X_test, Y_train, Y_test = train_test_split(pcm_df[x_list + list(total_eq_df.columns)].values, pcm_df[[target]].values, test_size=0.1, random_state=321, shuffle=False)

    x_scaler = MinMaxScaler(feature_range=(1,100))
    x_scaler.fit(X_train)
    X_train = x_scaler.transform(X_train)
    X_test = x_scaler.transform(X_test)


    y_scaler = MinMaxScaler(feature_range=(1,100))
    y_scaler.fit(Y_train)
    Y_train = y_scaler.transform(Y_train)
    Y_test = y_scaler.transform(Y_test)

    model_LR = RandomForestRegressor(random_state=777)
    model_LR.fit(X_train,Y_train)

    Y_test = y_scaler.inverse_transform(Y_test).reshape(-1)
    pred = y_scaler.inverse_transform(model_LR.predict(X_test).reshape(-1,1)).reshape(-1)
    
    r2score = r2_score(Y_test, pred)
    print('R-squared: ', r2score)
    mse = mean_squared_error(Y_test, pred)
    print('MSE: ', mse)
    mae = mean_absolute_error(Y_test, pred)
    print('MAE: ', mae)
    rmse = sqrt(mean_squared_error(Y_test, pred))
    print('RMSE: ', rmse)
    mape = np.mean(np.abs(pred - Y_test)/np.abs(Y_test))
    print('MAPE: ' ,'{:.2f}%'.format(mape*100))
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=pd.RangeIndex(len(Y_train), len(Y_train)+len(Y_test)), y=pred, name='pred'))
    fig.add_trace(go.Scatter(x=pd.RangeIndex(len(Y_train), len(Y_train)+len(Y_test)), y=Y_test, name='test'))
#     fig.add_trace(go.Scatter(x=pd.RangeIndex(0, len(Y_train)), y=y_scaler.inverse_transform(Y_train).reshape(-1), name='train'))
    fig.show()
    
    display(pd.DataFrame(model_LR.feature_importances_, index = x_list+ list(total_eq_df.columns)).sort_values(0, ascending=False))
    return Y_test, pred

In [ ]:
f_pcm_df = pcm_df[~pcm_df['DEVICE'].isin(['KTA9', 'KTAJ'])]
f_iqc_df = iqc_df[iqc_df['LOT_NO'].isin(f_pcm_df['LOT_NO'])]
f_iqc_df['ROUTE'] = [item[3:] for item in f_iqc_df['LPT']]
f_iqc_df = f_iqc_df.sort_values('ROUTE')
f_iqc_df = f_iqc_df[~(f_iqc_df['LOT_NO'] == '1102100')]
f_iqc_df['MAIN_EQ'] = f_iqc_df['ROUTE'] + '_' + f_iqc_df['MAIN_EQ'] + '_' + f_iqc_df['EDC_CHAMBER'].fillna('')

# Feature Selection에서 사용한 Route list up
route_list = ['1953', '4800', '6400']
# Feature Selection 결과
x_list = ['1953_3Y_AVG', '1953_CD15AS_MIN', '1953_MAG-X_MAX', '1953_R/R_MAX',
       '1953_W_ROT_AVG', '1953_Y_MAX_MAX', '4800_3X_MIN', '4800_3Y_AVG',
       '6400_2X_MAX', '6400_R/R_MAX'
]
true, pred = make_n_val_model(f_pcm_df, f_iqc_df, route_list, x_list, 'AVG')
# make_n_val_model(f_pcm_df, f_iqc_df, route_list, x_list, 'MIN')
# make_n_val_model(f_pcm_df, f_iqc_df, route_list, x_list, 'MAX')